# 1986. Minimum Number of Work Sessions to Finish the Tasks

## Topic Alignment
- Scheduling tasks into fixed-length sessions mirrors ML pipeline batching; bitmask DFS with memo finds minimal sessions respecting capacity limits.

## Metadata 摘要
- Source: https://leetcode.com/problems/minimum-number-of-work-sessions-to-finish-the-tasks/
- Tags: DFS, Bitmask, Memoization
- Difficulty: Medium
- Priority: High

## Problem Statement 原题描述
You are given n tasks where task durations are tasks[i] and a sessionTime limit. Each work session can handle tasks up to sessionTime, sequentially. Find the minimum number of sessions needed to finish all tasks if tasks cannot be split across sessions.

## Progressive Hints
- Hint 1: n <= 14 allows bitmask DP/DFS.
- Hint 2: Sort tasks descending to place longer jobs first.
- Hint 3: DFS tries to fit tasks into current session or start a new one; memoize (mask, current_time).

## Solution Overview
Use DFS with memoization: state (mask, remaining_time) denotes tasks already scheduled and remaining capacity in current session. Branch by selecting next unscheduled task. If duration <= remaining_time, schedule in current session; otherwise start a new session (increment count). Memoize minimal sessions from each state.

## Detailed Explanation
1. Sort tasks descending.
2. DFS returns minimum sessions needed given mask and remaining time in current session.
3. If mask == full_mask return 1 if remaining_time != sessionTime else 0 (no new session).
4. Iterate tasks; if unused, try to place. If duration <= remaining_time, result = dfs(mask|bit, remaining_time - duration).
5. Also consider starting new session: 1 + dfs(mask|bit, sessionTime - duration).
6. Take minimum of feasible choices, memoize by (mask, remaining_time).

## Complexity Trade-off Table
| Approach | Time | Space | Notes |
| --- | --- | --- | --- |
| DFS + memo | O(2^n * n) | O(2^n * sessionTime) | n <= 14 feasible |
| Greedy | - | - | Fails on counterexamples |

In [ ]:
from functools import lru_cache
from typing import List

class Solution:
    def minSessions(self, tasks: List[int], sessionTime: int) -> int:
        tasks.sort(reverse=True)
        n = len(tasks)
        full_mask = (1 << n) - 1
        @lru_cache(None)
        def dfs(mask: int, remaining: int) -> int:
            if mask == full_mask:
                return 0 if remaining == sessionTime else 1
            best = float('inf')
            for i in range(n):
                bit = 1 << i
                if mask & bit:
                    continue
                duration = tasks[i]
                if duration <= remaining:
                    best = min(best, dfs(mask | bit, remaining - duration))
                else:
                    best = min(best, 1 + dfs(mask | bit, sessionTime - duration))
                if best == 0:
                    break
            return best
        return dfs(0, sessionTime)

In [ ]:
tests = [
    ([1,2,3], 3, 2),
    ([3,3,3], 3, 3),
    ([1,2,3,4,5], 15, 1),
    ([1,2,3,4,5], 5, 4)
]
solver = Solution()
for tasks, session, expected in tests:
    assert solver.minSessions(tasks[:], session) == expected
print('All tests passed.')

## Complexity Analysis
- Time: O(2^n * n) since each subset and next task combination is considered.
- Space: O(2^n * sessionTime) in memo; recursion depth up to n.

## Edge Cases & Pitfalls
- Sorting descending reduces branching because large tasks fill sessions quickly.
- Remember to count ongoing session if remaining != sessionTime when all tasks scheduled.
- Avoid exploring equivalent states by keeping tasks sorted and always considering first unused task positions.

## Follow-up Variants
- Return actual session groupings along with count.
- Allow preemption (splitting tasks); transform into bin packing with fractional items.
- Add task transition costs or dependencies requiring topological ordering.

## Takeaways
- Bitmask memoization is powerful for n ≤ 14 scheduling problems.
- Sorting tasks descending prunes large parts of search space.
- Representing states with remaining capacity keeps recursion decisions local.

## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| LC 1723 | Find Minimum Time to Finish All Jobs | Bitmask DP assigning jobs to workers |
| LC 698 | Partition to K Equal Sum Subsets | Bitmask DFS with target sum |
| LC 875 | Koko Eating Bananas | Binary search on time (contrast) |